In [389]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast

In [390]:
# creamos las variables para las Rutas relativas a los archivos JSON
ruta_Output_Steam_Games = os.path.join('Datasets-Limpios', 'output_games.parquet')
ruta_AUS_User_Items = os.path.join('Datasets-Limpios', 'user_items.parquet')
ruta_AUS_User_Reviews = os.path.join('Datasets-Limpios', 'user_reviews.parquet')

In [391]:
games = pd.read_parquet(ruta_Output_Steam_Games)
items = pd.read_parquet(ruta_AUS_User_Items)
reviews = pd.read_parquet(ruta_AUS_User_Reviews)

In [392]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ItemId      58431 non-null  int32 
 1   Recommend   58431 non-null  bool  
 2   Review      58431 non-null  object
 3   UserId      58431 non-null  object
 4   YearPosted  58431 non-null  int32 
dtypes: bool(1), int32(2), object(2)
memory usage: 1.4+ MB


In [393]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094092 entries, 0 to 5094091
Data columns (total 5 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   ItemId           int32 
 1   ItemName         object
 2   PlaytimeForever  int32 
 3   UserId           object
 4   ItemsCount       int32 
dtypes: int32(3), object(2)
memory usage: 136.0+ MB


In [394]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31994 entries, 0 to 31993
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Genres       31994 non-null  object        
 1   ItemName     31994 non-null  object        
 2   ReleaseDate  29686 non-null  datetime64[ns]
 3   Price        31994 non-null  float64       
 4   ItemId       31994 non-null  int32         
 5   Developer    31994 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(3)
memory usage: 1.3+ MB


    Distribución de Datos:
        Examina la distribución de las similitudes de coseno entre los elementos del conjunto de datos. Esto puede darte una idea de cómo están distribuidas las similitudes y si hay algún patrón interesante.

    Estadísticas Descriptivas:
        Calcula estadísticas descriptivas como la media, mediana, desviación estándar, etc., de las similitudes de coseno. Esto puede ayudarte a entender la tendencia central y la dispersión de las similitudes.

    Visualizaciones:
        Utiliza visualizaciones como histogramas, gráficos de dispersión y cajas para explorar la variabilidad y la distribución de las similitudes de coseno.

    Análisis de Outliers:
        Identifica y analiza posibles valores atípicos en las similitudes. Los valores atípicos pueden afectar significativamente el rendimiento del modelo y es importante entender su impacto.

    Variabilidad entre Grupos:
        Si tienes información sobre diferentes grupos de usuarios o elementos, investiga la variabilidad de las similitudes dentro y entre estos grupos. Puede ser útil entender si hay ciertas categorías que tienen similitudes más altas o más bajas en promedio.

    Matriz de Correlación (Opcional):
        Aunque la matriz de correlación es más comúnmente utilizada en el contexto de variables numéricas, podrías explorar la posibilidad de calcular la matriz de correlación entre las similitudes de coseno de diferentes elementos. Esto puede revelar patrones interesantes en la similitud entre elementos.

    Análisis de Sesgo:
        Examina si hay sesgo en las recomendaciones hacia ciertos usuarios o elementos. Puedes analizar si ciertos grupos están recibiendo recomendaciones de manera desproporcionada.

    Validación Cruzada:
        Si tienes un conjunto de datos grande, considera realizar validación cruzada para evaluar la estabilidad y robustez del modelo de recomendación basado en similitud de coseno.

Recuerda que el EDA es un proceso iterativo y depende en gran medida de la naturaleza específica de tus datos y del problema de recomendación que estás abordando. A medida que explores los datos, es posible que surjan nuevas preguntas y consideraciones que puedan guiar tu análisis.

Si es un sistema de recomendación item-item:

    def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.


In [395]:
modelo_df = games.loc[:, ["Genres", "ItemId",'ItemName']]

In [396]:
modelo_df.head()

,Genres,ItemId,ItemName
0,"[Action, Casual, Indie, Simulation, Strategy]",761140,Lost Summoner Kitty
1,"[Free to Play, Indie, RPG, Strategy]",643980,Ironbound
2,"[Casual, Free to Play, Indie, Simulation, Sports]",670290,Real Pool 3D - Poolians
3,"[Action, Adventure, Casual]",767400,弹炸人2222
4,"[Action, Indie, Casual, Sports]",773570,Log Challenge


In [397]:
modelo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31994 entries, 0 to 31993
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Genres    31994 non-null  object
 1   ItemId    31994 non-null  int32 
 2   ItemName  31994 non-null  object
dtypes: int32(1), object(2)
memory usage: 625.0+ KB


In [398]:
modelo_df['Genres'].dtype

dtype('O')

In [399]:
# Asignar la columna 'Genres' a una variable
genres_variable = modelo_df['Genres'].copy()
# Convertir a cadena antes de reemplazar los corchetes
genres_variable = genres_variable.apply(lambda x: str(x).replace('[', '').replace(']', ''))
# Eliminar comillas y agregar comas entre los géneros
genres_variable = genres_variable.apply(lambda x: ', '.join(filter(None, map(str.strip, x.split("'")))) if isinstance(x, str) else x)
# Reemplazar la columna 'Genres' en el DataFrame original
modelo_df['Genres'] = genres_variable

# Imprimir la columna después de reemplazar
print(modelo_df['Genres'])

0              Action, Casual, Indie, Simulation, Strategy
1                       Free to Play, Indie, RPG, Strategy
2          Casual, Free to Play, Indie, Simulation, Sports
3                                Action, Adventure, Casual
4                            Action, Indie, Casual, Sports
                               ...                        
31989                  Casual, Indie, Simulation, Strategy
31990                              Casual, Indie, Strategy
31991                            Indie, Racing, Simulation
31992                                        Casual, Indie
31993    Early Access, Adventure, Indie, Action, Simula...
Name: Genres, Length: 31994, dtype: object


In [400]:
#vamos a cambiar unos errores que hay en la columna Genres
# como vemos hay unos errores de caracteres, que vamos a solucionar de la siguiente manera
modelo_df['Genres'] = modelo_df['Genres'].replace('Design &amp; Illustration', 'Design & Illustration')
modelo_df['Genres'] = modelo_df['Genres'].replace('Animation &amp; Modeling', 'Animation & Modeling')

In [401]:
print(modelo_df['Genres'].unique())

['Action, Casual, Indie, Simulation, Strategy'
 'Free to Play, Indie, RPG, Strategy'
 'Casual, Free to Play, Indie, Simulation, Sports' ...
 'Action, Massively Multiplayer, Strategy'
 'Video Production, Utilities, Web Publishing'
 'Early Access, Adventure, Indie, Action, Simulation']


Porque consideramos poner dummies

In [402]:
# Imprimir algunas entradas problemáticas
for x in modelo_df['Genres']:
    try:
        ast.literal_eval(x)
    except ValueError:
        print(f'Error en la entrada: {x}')

Error en la entrada: Action, Casual, Indie, Simulation, Strategy


SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
# Convertir la cadena a una lista usando split
modelo_df['Genres'] = modelo_df['Genres'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)

In [ ]:
# Obtener columnas de dummies para los géneros
generos_dummies = pd.get_dummies(modelo_df['Genres'].apply(pd.Series).stack()).sum(level=0)

C:\Users\frank\AppData\Local\Temp\ipykernel_21940\337183157.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  generos_dummies = pd.get_dummies(modelo_df['Genres'].apply(pd.Series).stack()).sum(level=0)


In [ ]:
generos_dummies.head(1)

,,Accounting,Action,Adventure,Animation & Modeling,Animation &amp; Modeling,Audio Production,Casual,Design & Illustration,Design &amp; Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0


In [ ]:
generos_dummies.columns

Index(['', 'Accounting', 'Action', 'Adventure', 'Animation & Modeling',
       'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design & Illustration', 'Design &amp; Illustration', 'Early Access',
       'Education', 'Free to Play', 'Indie', 'Massively Multiplayer',
       'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training',
       'Sports', 'Strategy', 'Utilities', 'Video Production',
       'Web Publishing'],
      dtype='object')

In [ ]:
generos_dummies.drop(columns=['Animation &amp; Modeling','Design &amp; Illustration',''], inplace=True)

In [ ]:
# Concatenar las columnas de dummies al DataFrame original
modelo_df = pd.concat([modelo_df, generos_dummies], axis=1)

In [ ]:
modelo_df.head(1)

,Genres,ItemId,ItemName,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,"Action, Casual, Indie, Simulation, Strategy",761140,Lost Summoner Kitty,0,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0


In [ ]:
modelo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31994 entries, 0 to 31993
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Genres                 31994 non-null  object
 1   ItemId                 31994 non-null  int32 
 2   ItemName               31994 non-null  object
 3   Accounting             31994 non-null  uint8 
 4   Action                 31994 non-null  uint8 
 5   Adventure              31994 non-null  uint8 
 6   Animation & Modeling   31994 non-null  uint8 
 7   Audio Production       31994 non-null  uint8 
 8   Casual                 31994 non-null  uint8 
 9   Design & Illustration  31994 non-null  uint8 
 10  Early Access           31994 non-null  uint8 
 11  Education              31994 non-null  uint8 
 12  Free to Play           31994 non-null  uint8 
 13  Indie                  31994 non-null  uint8 
 14  Massively Multiplayer  31994 non-null  uint8 
 15  Photo Editing      

Modelo de recomendacion item- item

In [ ]:
'''similarity_matrix = cosine_similarity(X)  
df_similarity = pd.DataFrame(similarity_matrix, index=generos_dummies['id'], columns=df_dummies['id'])
Salida={}
for columna in df_similarity.columns:
    prueba=df_similarity[columna].nlargest(6).index[1:].to_list()
    prueba3=df_Juegos['app_name'][df_Juegos['id'].isin(prueba)].to_list()
    Salida.update({columna:prueba3})'''

In [45]:
cv = CountVectorizer()

In [46]:
cv.fit_transform(modelo_df['Genres']).toarray().shape

AttributeError: 'numpy.ndarray' object has no attribute 'lower'